# Executive Summary: ICU Patient Prioritization Report

## For: ICU Medical Management Team
## Date: 2026-02-10
## Prepared by: Healthcare Analytics Team

---

## Purpose
This report provides data-driven insights to help ICU medical managers prioritize patient care based on risk factors, diagnosis severity, and resource requirements identified from historical patient data.

## Data Source
- **Dataset:** MIMIC-IV Clinical Database Demo v2.2
- **Sample:** 100 patients, 140 ICU stays, 275 hospital admissions
- **Scope:** Patient demographics, diagnoses, ICU admissions, vital signs, and outcomes

---

## Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 80)
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("Set2")

print("Analytics environment ready")

## Load Data

In [ ]:
# Define paths
DATA_PATH_HOSP = '../../../../mimic-iv-demo/mimic-iv-clinical-database-demo-2.2/hosp/'
DATA_PATH_ICU = '../../../../mimic-iv-demo/mimic-iv-clinical-database-demo-2.2/icu/'

# Load necessary tables
patients = pd.read_csv(DATA_PATH_HOSP + 'patients.csv')
admissions = pd.read_csv(DATA_PATH_HOSP + 'admissions.csv')
diagnoses = pd.read_csv(DATA_PATH_HOSP + 'diagnoses_icd.csv')
icd_dict = pd.read_csv(DATA_PATH_HOSP + 'd_icd_diagnoses.csv')
icustays = pd.read_csv(DATA_PATH_ICU + 'icustays.csv')

print("Data loaded successfully")

## Data Preparation

In [ ]:
# Prepare ICU stays data
icustays['intime'] = pd.to_datetime(icustays['intime'])
icustays['outtime'] = pd.to_datetime(icustays['outtime'])
icustays['icu_los_days'] = (icustays['outtime'] - icustays['intime']).dt.total_seconds() / (24 * 3600)

# Prepare admissions data
admissions['admittime'] = pd.to_datetime(admissions['admittime'])
admissions['dischtime'] = pd.to_datetime(admissions['dischtime'])
admissions['hospital_los_days'] = (admissions['dischtime'] - admissions['admittime']).dt.total_seconds() / (24 * 3600)
admissions['had_icu'] = admissions['hadm_id'].isin(icustays['hadm_id'])

# Merge diagnoses with descriptions
diagnoses_full = diagnoses.merge(
    icd_dict[['icd_code', 'icd_version', 'long_title']], 
    on=['icd_code', 'icd_version'], 
    how='left'
)

# Create comprehensive ICU dataset
icu_complete = icustays.merge(
    patients[['subject_id', 'gender', 'anchor_age', 'dod']], 
    on='subject_id', 
    how='left'
).merge(
    admissions[['hadm_id', 'admission_type', 'hospital_expire_flag', 'hospital_los_days']], 
    on='hadm_id', 
    how='left'
)

# Get ICU diagnoses
icu_admissions = icustays['hadm_id'].unique()
icu_diagnoses = diagnoses_full[diagnoses_full['hadm_id'].isin(icu_admissions)].copy()

print(f"Data prepared for {len(icu_complete)} ICU stays")

---
# EXECUTIVE SUMMARY
---

## 1. Critical Findings Overview

In [ ]:
# Calculate key metrics
total_admissions = len(admissions)
icu_admissions_count = admissions['had_icu'].sum()
icu_rate = (icu_admissions_count / total_admissions) * 100

icu_mortality = (icu_complete['hospital_expire_flag'].sum() / len(icu_complete)) * 100
non_icu_mortality = (admissions[~admissions['had_icu']]['hospital_expire_flag'].sum() / 
                     (~admissions['had_icu']).sum()) * 100
mortality_multiplier = icu_mortality / non_icu_mortality

avg_icu_los = icu_complete['icu_los_days'].mean()
avg_hospital_los_icu = icu_complete['hospital_los_days'].mean()

print("="*80)
print("CRITICAL METRICS FOR ICU MANAGEMENT")
print("="*80)
print(f"\n[ICU UTILIZATION]")
print(f"  ICU admission rate: {icu_rate:.1f}% of all admissions")
print(f"  Total ICU stays analyzed: {len(icu_complete)}")

print(f"\n[MORTALITY RISK]")
print(f"  ICU patient mortality rate: {icu_mortality:.1f}%")
print(f"  Non-ICU patient mortality rate: {non_icu_mortality:.1f}%")
print(f"  >> ICU patients are {mortality_multiplier:.1f}x MORE LIKELY to die")

print(f"\n[RESOURCE REQUIREMENTS]")
print(f"  Average ICU length of stay: {avg_icu_los:.1f} days")
print(f"  Average total hospital stay (ICU patients): {avg_hospital_los_icu:.1f} days")

print("\n" + "="*80)

In [ ]:
# Visualize key metrics
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# 1. ICU Admission Rate
axes[0].bar(['Non-ICU', 'ICU'], [100-icu_rate, icu_rate], color=['lightgreen', 'crimson'], edgecolor='black')
axes[0].set_ylabel('Percentage of Admissions (%)', fontsize=11)
axes[0].set_title('ICU Admission Rate', fontsize=13, fontweight='bold')
axes[0].set_ylim(0, 100)
for i, v in enumerate([100-icu_rate, icu_rate]):
    axes[0].text(i, v + 2, f'{v:.1f}%', ha='center', fontsize=12, fontweight='bold')

# 2. Mortality Comparison
axes[1].bar(['Non-ICU', 'ICU'], [non_icu_mortality, icu_mortality], 
            color=['lightblue', 'darkred'], edgecolor='black')
axes[1].set_ylabel('Mortality Rate (%)', fontsize=11)
axes[1].set_title('Mortality Risk Comparison', fontsize=13, fontweight='bold')
for i, v in enumerate([non_icu_mortality, icu_mortality]):
    axes[1].text(i, v + 1, f'{v:.1f}%', ha='center', fontsize=12, fontweight='bold')

# 3. Length of Stay
avg_los_non_icu = admissions[~admissions['had_icu']]['hospital_los_days'].mean()
axes[2].bar(['Non-ICU', 'ICU Patients'], [avg_los_non_icu, avg_hospital_los_icu], 
            color=['lightgreen', 'orange'], edgecolor='black')
axes[2].set_ylabel('Average Hospital Stay (days)', fontsize=11)
axes[2].set_title('Hospital Length of Stay', fontsize=13, fontweight='bold')
for i, v in enumerate([avg_los_non_icu, avg_hospital_los_icu]):
    axes[2].text(i, v + 0.5, f'{v:.1f}d', ha='center', fontsize=12, fontweight='bold')

plt.suptitle('KEY PERFORMANCE INDICATORS', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

---
## 2. HIGH-RISK PATIENT IDENTIFICATION
---

### 2.1 Age-Based Risk Stratification

In [ ]:
# Create age groups and analyze mortality
icu_complete['age_group'] = pd.cut(icu_complete['anchor_age'], 
                                     bins=[0, 18, 35, 50, 65, 80, 100], 
                                     labels=['0-18', '19-35', '36-50', '51-65', '66-80', '80+'])

mortality_by_age = icu_complete.groupby('age_group')['hospital_expire_flag'].agg(['sum', 'count', 'mean'])
mortality_by_age.columns = ['Deaths', 'Total Patients', 'Mortality Rate']
mortality_by_age['Mortality Rate'] = mortality_by_age['Mortality Rate'] * 100

print("AGE-BASED MORTALITY RISK IN ICU:")
print("="*70)
print(mortality_by_age)
print("\n" + "="*70)
print("\nCLINICAL RECOMMENDATION:")
highest_risk_age = mortality_by_age['Mortality Rate'].idxmax()
print(f">> PRIORITIZE: Age group {highest_risk_age} shows highest mortality risk")
print(f">> Implement enhanced monitoring protocols for elderly patients (65+)")

In [ ]:
# Visualize age-based risk
fig, ax = plt.subplots(figsize=(12, 6))

colors = ['green', 'yellowgreen', 'yellow', 'orange', 'orangered', 'darkred']
bars = ax.bar(mortality_by_age.index.astype(str), mortality_by_age['Mortality Rate'], 
              color=colors, edgecolor='black', linewidth=1.5)

ax.set_ylabel('Mortality Rate (%)', fontsize=12, fontweight='bold')
ax.set_xlabel('Age Group', fontsize=12, fontweight='bold')
ax.set_title('PATIENT RISK STRATIFICATION BY AGE GROUP', fontsize=14, fontweight='bold')
ax.axhline(y=icu_mortality, color='red', linestyle='--', linewidth=2, label=f'Overall ICU Mortality ({icu_mortality:.1f}%)')
ax.legend(fontsize=11)
ax.grid(axis='y', alpha=0.3)

# Add value labels and patient counts
for i, (bar, rate, count) in enumerate(zip(bars, mortality_by_age['Mortality Rate'], mortality_by_age['Total Patients'])):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height + 1,
            f'{rate:.1f}%\n(n={int(count)})',
            ha='center', va='bottom', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

### 2.2 ICU Length of Stay and Mortality

In [ ]:
# Analyze relationship between ICU LOS and mortality
survivors = icu_complete[icu_complete['hospital_expire_flag'] == 0]['icu_los_days']
non_survivors = icu_complete[icu_complete['hospital_expire_flag'] == 1]['icu_los_days']

print("ICU LENGTH OF STAY BY OUTCOME:")
print("="*70)
print(f"\nSurvivors:")
print(f"  Average ICU stay: {survivors.mean():.2f} days")
print(f"  Median ICU stay: {survivors.median():.2f} days")

print(f"\nNon-Survivors:")
print(f"  Average ICU stay: {non_survivors.mean():.2f} days")
print(f"  Median ICU stay: {non_survivors.median():.2f} days")

print("\n" + "="*70)
print("\nCLINICAL INSIGHT:")
if non_survivors.mean() > survivors.mean():
    print(">> Non-survivors have LONGER ICU stays on average")
    print(">> Implement early intervention protocols for patients with extended stays")
else:
    print(">> Non-survivors have SHORTER ICU stays (may indicate rapid deterioration)")
    print(">> Focus on early identification and aggressive treatment")

In [ ]:
# Visualize LOS by outcome
fig, ax = plt.subplots(figsize=(10, 6))

bp = ax.boxplot([survivors, non_survivors], labels=['Survived', 'Died'], 
                 patch_artist=True, widths=0.6)

# Color the boxes
bp['boxes'][0].set_facecolor('lightgreen')
bp['boxes'][1].set_facecolor('lightcoral')

for box in bp['boxes']:
    box.set_edgecolor('black')
    box.set_linewidth(2)

ax.set_ylabel('ICU Length of Stay (days)', fontsize=12, fontweight='bold')
ax.set_xlabel('Outcome', fontsize=12, fontweight='bold')
ax.set_title('ICU STAY DURATION BY PATIENT OUTCOME', fontsize=14, fontweight='bold')
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

---
## 3. CRITICAL DIAGNOSES REQUIRING PRIORITIZATION
---

### 3.1 Highest Mortality Diagnoses

In [ ]:
# Merge ICU diagnoses with outcomes
diag_with_outcomes = icu_diagnoses.merge(
    admissions[['hadm_id', 'hospital_expire_flag']], 
    on='hadm_id', 
    how='left'
)

# Calculate mortality by diagnosis
mortality_by_diagnosis = diag_with_outcomes.groupby(['icd_code', 'long_title']).agg({
    'hospital_expire_flag': ['sum', 'count', 'mean']
}).reset_index()

mortality_by_diagnosis.columns = ['icd_code', 'diagnosis', 'deaths', 'total_cases', 'mortality_rate']
mortality_by_diagnosis['mortality_rate'] = mortality_by_diagnosis['mortality_rate'] * 100

# Filter for diagnoses with at least 3 cases for statistical relevance
high_risk_diagnoses = mortality_by_diagnosis[mortality_by_diagnosis['total_cases'] >= 3]
high_risk_diagnoses = high_risk_diagnoses.sort_values('mortality_rate', ascending=False)

print("TOP 10 HIGHEST MORTALITY DIAGNOSES IN ICU:")
print("="*100)
for idx, row in high_risk_diagnoses.head(10).iterrows():
    diagnosis_short = row['diagnosis'][:70] + '...' if len(row['diagnosis']) > 70 else row['diagnosis']
    print(f"{row['mortality_rate']:>6.1f}% mortality | {int(row['deaths'])}/{int(row['total_cases'])} cases | {diagnosis_short}")
print("\n" + "="*100)

In [ ]:
# Visualize top high-risk diagnoses
fig, ax = plt.subplots(figsize=(14, 8))

top_10_risk = high_risk_diagnoses.head(10)
top_10_risk['short_diagnosis'] = top_10_risk['diagnosis'].str[:50] + '...'

# Color gradient based on mortality rate
colors = plt.cm.Reds(top_10_risk['mortality_rate'] / 100)

bars = ax.barh(range(len(top_10_risk)), top_10_risk['mortality_rate'], 
               color=colors, edgecolor='black', linewidth=1.5)

ax.set_yticks(range(len(top_10_risk)))
ax.set_yticklabels(top_10_risk['short_diagnosis'])
ax.set_xlabel('Mortality Rate (%)', fontsize=12, fontweight='bold')
ax.set_title('TOP 10 DIAGNOSES WITH HIGHEST MORTALITY IN ICU', fontsize=14, fontweight='bold')
ax.invert_yaxis()
ax.grid(axis='x', alpha=0.3)

# Add value labels with case counts
for i, (rate, cases) in enumerate(zip(top_10_risk['mortality_rate'], top_10_risk['total_cases'])):
    ax.text(rate + 2, i, f'{rate:.1f}% (n={int(cases)})', va='center', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

### 3.2 Most Common ICU Diagnoses

In [ ]:
# Most frequent diagnoses in ICU
frequent_diagnoses = icu_diagnoses.groupby(['icd_code', 'long_title']).size().reset_index(name='count')
frequent_diagnoses = frequent_diagnoses.sort_values('count', ascending=False)

print("TOP 10 MOST COMMON ICU DIAGNOSES:")
print("="*90)
for idx, row in frequent_diagnoses.head(10).iterrows():
    diagnosis_short = row['long_title'][:70] + '...' if len(row['long_title']) > 70 else row['long_title']
    print(f"{int(row['count']):>3} cases | {diagnosis_short}")
print("\n" + "="*90)
print("\nCLINICAL RECOMMENDATION:")
print(">> Ensure adequate staffing and resources for most common conditions")
print(">> Develop standardized care protocols for high-volume diagnoses")

---
## 4. RESOURCE ALLOCATION INSIGHTS
---

### 4.1 ICU Stay Duration Categories

In [ ]:
# Categorize ICU stays by duration
short_stay = (icu_complete['icu_los_days'] <= 1).sum()
medium_stay = ((icu_complete['icu_los_days'] > 1) & (icu_complete['icu_los_days'] <= 7)).sum()
long_stay = (icu_complete['icu_los_days'] > 7).sum()

stay_categories = pd.DataFrame({
    'Category': ['Short Stay\n(≤ 1 day)', 'Medium Stay\n(1-7 days)', 'Long Stay\n(> 7 days)'],
    'Count': [short_stay, medium_stay, long_stay],
    'Percentage': [short_stay/len(icu_complete)*100, 
                  medium_stay/len(icu_complete)*100, 
                  long_stay/len(icu_complete)*100]
})

print("ICU STAY DURATION DISTRIBUTION:")
print("="*70)
print(stay_categories.to_string(index=False))
print("\n" + "="*70)
print("\nRESOURCE PLANNING INSIGHT:")
print(f">> {stay_categories.iloc[2]['Percentage']:.1f}% of patients require extended ICU care (>7 days)")
print(">> Plan bed capacity and staffing for long-term ICU patients")
print(">> Consider step-down units for patients transitioning out of ICU")

In [ ]:
# Visualize stay categories
fig, ax = plt.subplots(figsize=(10, 7))

colors_pie = ['lightgreen', 'gold', 'orangered']
explode = (0.05, 0.05, 0.1)  # Emphasize long stay

wedges, texts, autotexts = ax.pie(stay_categories['Count'], 
                                    labels=stay_categories['Category'],
                                    autopct='%1.1f%%',
                                    colors=colors_pie,
                                    explode=explode,
                                    startangle=90,
                                    textprops={'fontsize': 12, 'fontweight': 'bold'})

for autotext in autotexts:
    autotext.set_color('white')
    autotext.set_fontsize(13)

ax.set_title('ICU STAY DURATION DISTRIBUTION', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

---
## 5. ACTIONABLE RECOMMENDATIONS FOR ICU MANAGEMENT
---

### Priority Actions Based on Data Analysis

In [ ]:
print("="*90)
print("CLINICAL RECOMMENDATIONS FOR ICU PATIENT PRIORITIZATION")
print("="*90)

print("\n[1] HIGH-RISK PATIENT IDENTIFICATION")
print("    Action Items:")
print(f"    - Implement enhanced monitoring for patients aged 65+ (highest mortality risk)")
print(f"    - ICU patients are {mortality_multiplier:.1f}x more likely to die than non-ICU patients")
print("    - Establish rapid response protocols for early deterioration signs")

print("\n[2] DIAGNOSIS-SPECIFIC PROTOCOLS")
print("    Action Items:")
top_risk_diag = high_risk_diagnoses.iloc[0]
print(f"    - Highest mortality diagnosis: {top_risk_diag['diagnosis'][:60]}...")
print(f"      ({top_risk_diag['mortality_rate']:.1f}% mortality rate)")
print("    - Develop specialized care pathways for high-mortality conditions")
print("    - Ensure intensivist availability for critical diagnoses")

print("\n[3] RESOURCE ALLOCATION")
print("    Action Items:")
print(f"    - Average ICU stay: {avg_icu_los:.1f} days - plan staffing accordingly")
print(f"    - {stay_categories.iloc[2]['Percentage']:.1f}% require extended care (>7 days)")
print("    - Allocate dedicated resources for long-stay ICU patients")
print("    - Consider step-down unit capacity for ICU overflow")

print("\n[4] QUALITY IMPROVEMENT INITIATIVES")
print("    Action Items:")
print("    - Implement daily ICU rounds with multidisciplinary team")
print("    - Establish mortality review process for high-risk diagnoses")
print("    - Track and optimize ICU length of stay to improve patient flow")
print("    - Develop early warning score system for patient deterioration")

print("\n[5] MONITORING AND METRICS")
print("    Key Performance Indicators to Track:")
print(f"    - ICU mortality rate (current: {icu_mortality:.1f}%)")
print(f"    - Average ICU LOS (current: {avg_icu_los:.1f} days)")
print("    - ICU readmission rate within 48 hours")
print("    - Bed occupancy rate and turnover time")

print("\n" + "="*90)
print("\nREPORT PREPARED BY: Healthcare Analytics Team")
print("DATE: 2026-02-10")
print("DATA SOURCE: MIMIC-IV Clinical Database Demo v2.2")
print("="*90)

---
## APPENDIX: Summary Statistics

In [ ]:
# Create comprehensive summary table
summary_stats = pd.DataFrame({
    'Metric': [
        'Total Hospital Admissions',
        'ICU Admissions',
        'ICU Admission Rate',
        'Average Patient Age (ICU)',
        'ICU Mortality Rate',
        'Non-ICU Mortality Rate',
        'Average ICU Length of Stay',
        'Average Hospital LOS (ICU patients)',
        'Short Stays (≤1 day)',
        'Medium Stays (1-7 days)',
        'Long Stays (>7 days)'
    ],
    'Value': [
        f"{total_admissions}",
        f"{icu_admissions_count}",
        f"{icu_rate:.1f}%",
        f"{icu_complete['anchor_age'].mean():.1f} years",
        f"{icu_mortality:.1f}%",
        f"{non_icu_mortality:.1f}%",
        f"{avg_icu_los:.2f} days",
        f"{avg_hospital_los_icu:.2f} days",
        f"{short_stay} ({short_stay/len(icu_complete)*100:.1f}%)",
        f"{medium_stay} ({medium_stay/len(icu_complete)*100:.1f}%)",
        f"{long_stay} ({long_stay/len(icu_complete)*100:.1f}%)"
    ]
})

print("\nSUMMARY STATISTICS TABLE:")
print("="*70)
print(summary_stats.to_string(index=False))
print("="*70)